In [2]:
import numpy as np
import pandas as pd
import mne

In [55]:
%%time

#make sure to change filename
edf = mne.io.read_raw_edf('n10.edf') 

header = ','.join(edf.ch_names)

#compare study start time with first txt timestamp in terms of seconds, then multiply by 500 and include as input
F = edf.get_data(start = 1800000)

D = F.transpose()
df = pd.DataFrame(D)

#copy corresponding values from columns spreadsheet
df.columns=['Fp2-F4','F4-C4','C4-P4','P4-O2','F8-T4','T4-T6','FP1-F3','F3-C3','C3-P3','P3-O1','F7-T3','T3-T5','C4-A1','ROC-LOC','EMG1-EMG2','ECG1-ECG2','DX1-DX2','SX1-SX2']

df['elapsed_seconds'] = [0.002*i for i in range(0, len(df.iloc[:,0]))]
df['epoch'] = df['elapsed_seconds'] // 30

df=df[['epoch','Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2','C4-A1','ROC-LOC','EMG1-EMG2','ECG1-ECG2','DX1-DX2','SX1-SX2']]

#Include feature extraction here:

df


Extracting EDF parameters from /Users/Ashok/OneDrive - Georgia Institute of Technology/Summer 2022/CS 4641/Project/n10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
CPU times: user 12.2 s, sys: 7.02 s, total: 19.2 s
Wall time: 25.2 s


,epoch,Fp2-F4,F4-C4,C4-P4,P4-O2,C4-A1,ROC-LOC,EMG1-EMG2,ECG1-ECG2,DX1-DX2,SX1-SX2
0,0.0,0.000008,0.000002,-0.000007,0.000002,-0.000006,-0.000033,1.443848e-06,0.000052,-1.678877e-06,-4.304029e-06
1,0.0,0.000008,0.000002,-0.000006,0.000002,-0.000005,-0.000033,1.138360e-05,0.000053,3.079290e-08,-4.747075e-06
2,0.0,0.000008,0.000002,-0.000005,0.000003,-0.000006,-0.000034,1.864844e-05,0.000050,2.260460e-06,-3.820706e-06
3,0.0,0.000009,0.000003,-0.000004,0.000005,-0.000006,-0.000034,1.329860e-05,0.000045,4.236000e-06,-2.068795e-06
4,0.0,0.000008,0.000004,-0.000003,0.000005,-0.000005,-0.000035,3.495633e-07,0.000039,5.311355e-06,-3.663004e-07
...,...,...,...,...,...,...,...,...,...,...,...
13252795,883.0,0.000015,-0.000014,-0.000039,-0.000016,-0.000014,-0.000338,-3.495633e-07,0.000043,-3.229593e-08,1.482575e-06
13252796,883.0,0.000017,-0.000013,-0.000039,-0.000017,-0.000016,-0.000279,1.671824e-07,0.000043,-1.221001e-07,1.373626e-06
13252797,883.0,0.000019,-0.000013,-0.000039,-0.000017,-0.000018,-0.000164,1.519840e-08,0.000042,-6.532156e-08,1.009328e-06
13252798,883.0,0.000022,-0.000013,-0.000039,-0.000018,-0.000021,-0.000013,-8.359122e-07,0.000042,9.087077e-08,4.530623e-07


In [57]:
#change filename
df2=pd.read_csv('n10.txt')[17:]

df2=df2.iloc[:,0].str.split('\t', expand=True)
df2.columns=[x for x in df2.iloc[0]]
df2=df2.iloc[1:]

#change to primary location associated with "SLEEP" event
df2=df2[df2['Location'] == 'ROC-LOC']

df2['epoch']=[i for i in range(0,len(df2.iloc[:,0]))]

#These need to be changed for each file - consult spreadsheet for gender and age
df2['condition'] = ['Normal' for i in range(0, len(df2.iloc[:,0]))]
df2['gender'] = ['M' for i in range(0, len(df2.iloc[:,0]))]
df2['age'] = [23 for i in range(0, len(df2.iloc[:,0]))]

df2=df2[['epoch', 'Sleep Stage', 'condition', 'gender', 'age']]

final=pd.merge(df2, df, how='left', on='epoch')
# final.to_csv('n10_clean.csv')
final

,epoch,Sleep Stage,condition,gender,age,Fp2-F4,F4-C4,C4-P4,P4-O2,C4-A1,ROC-LOC,EMG1-EMG2,ECG1-ECG2,DX1-DX2,SX1-SX2
0,0,W,Normal,M,23,0.000008,1.968864e-06,-0.000007,1.907814e-06,-0.000006,-0.000033,1.443848e-06,0.000052,-1.678877e-06,-4.304029e-06
1,0,W,Normal,M,23,0.000008,2.121490e-06,-0.000006,2.426740e-06,-0.000005,-0.000033,1.138360e-05,0.000053,3.079290e-08,-4.747075e-06
2,0,W,Normal,M,23,0.000008,2.487790e-06,-0.000005,3.281441e-06,-0.000006,-0.000034,1.864844e-05,0.000050,2.260460e-06,-3.820706e-06
3,0,W,Normal,M,23,0.000009,3.128816e-06,-0.000004,4.532967e-06,-0.000006,-0.000034,1.329860e-05,0.000045,4.236000e-06,-2.068795e-06
4,0,W,Normal,M,23,0.000008,3.739316e-06,-0.000003,5.448718e-06,-0.000005,-0.000035,3.495633e-07,0.000039,5.311355e-06,-3.663004e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12779995,851,S3,Normal,M,23,-0.000003,-3.815629e-07,-0.000001,1.022589e-06,0.000004,0.000158,2.887697e-07,0.000056,7.739901e-07,1.225709e-06
12779996,851,S3,Normal,M,23,-0.000003,-4.120879e-07,-0.000001,7.478632e-07,0.000003,0.000157,1.231071e-06,0.000056,6.410256e-07,1.404151e-06
12779997,851,S3,Normal,M,23,-0.000003,-4.731380e-07,-0.000001,3.510379e-07,0.000003,0.000157,1.839007e-06,0.000056,4.223034e-07,1.408975e-06
12779998,851,S3,Normal,M,23,-0.000003,-5.036630e-07,-0.000001,-1.068376e-07,0.000002,0.000157,1.717420e-06,0.000056,1.937218e-07,1.220289e-06
